In [1]:
!pip install eli5

In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

#imports to change "str_dict" to dict
from ast import literal_eval 
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data

mens_shoes_filtered.csv  shoe_prices.csv


In [0]:
df = pd.read_csv('data/mens_shoes_filtered.csv', low_memory=False)

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  X = df [ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]

In [8]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

**Model improvement**

In [9]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.47223572384038, 4.328288468270897)

**Feature engineering**

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [11]:
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [12]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [13]:
df.features[0]

'[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [15]:
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [16]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()))

len(keys)

476

In [17]:
def get_name_feat(key):
  return "feats_"+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map( lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull() ].shape[0] / df.shape[0] * 100

In [19]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

**Feature engineering**

In [0]:
df[get_name_feat('brand_cat')] = df[get_name_feat('brand')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('color_cat')] = df[get_name_feat('color')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('gender_cat')] = df[get_name_feat('gender')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('manufacturer part number_cat')] = df[get_name_feat('manufacturer part number')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('material_cat')] = df[get_name_feat('material')].map(lambda x: str(x).lower()).factorize()[0]

df[get_name_feat('sport_cat')] = df[get_name_feat('sport')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('style_cat')] = df[get_name_feat('style')].map(lambda x: str(x).lower()).factorize()[0]

for key in keys:
  df[get_name_feat(key)+'_cat'] = df[get_name_feat(key)].map(lambda x: str(x).lower()).factorize()[0]

In [29]:
df[ df.brand == df.feats_brand].shape

(8846, 531)

In [0]:
df['brand'] = df['brand'].map( lambda x: str(x).lower() )

In [30]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)

run_model(['brand_cat'], model)

(-57.33585993237942, 4.262401063578807)

In [47]:
feats_cat = [x for x in df.columns if "cat" in x]
feats_cat

['categories',
 'brand_cat',
 'feats_shoe category',
 'feats_location - city/state',
 'feats_recommended location',
 'feats_location - country',
 'feats_certifications and listings',
 'feats_multi pack indicator',
 'feats_clothing category',
 'feats_fabrication',
 'feats_catalog',
 'feats_brand_cat',
 'feats_color_cat',
 'feats_gender_cat',
 'feats_manufacturer part number_cat',
 'feats_material_cat',
 'feats_sport_cat',
 'feats_style_cat',
 'feats_fits over eyeglasses_cat',
 'feats_pocket_cat',
 'feats_watch band material_cat',
 'feats_labor_cat',
 'feats_military style_cat',
 'feats_gold karat_cat',
 'feats_chain length_cat',
 'feats_fit:_cat',
 'feats_pronation_cat',
 'feats_case type_cat',
 'feats_cushioning_cat',
 'feats_is polarized_cat',
 'feats_band width_cat',
 'feats_model_cat',
 'feats_bed size_cat',
 'feats_shoe category_cat',
 'feats_wheeled_cat',
 'feats_polarized lenses_cat',
 'feats_front style_cat',
 'feats_fit_cat',
 'feats_casing_cat',
 'feats_rx-able_cat',
 'feats_c

In [58]:
feats = ['brand_cat', get_name_feat('brand_cat'), get_name_feat('gender_cat'), get_name_feat('material_cat'), get_name_feat('style_cat'), get_name_feat('sport_cat')]
feats += [
# #  'feats_shoe category',
#  'feats_location - city/state',
#  'feats_recommended location',
#  'feats_location - country',
# #  'feats_certifications and listings',
#  'feats_multi pack indicator',
#  'feats_clothing category',
#  'feats_fabrication',
#  'feats_catalog',
#  'feats_brand_cat',
#  'feats_color_cat',
#  'feats_gender_cat',
#  'feats_manufacturer part number_cat',
#  'feats_material_cat',
#  'feats_sport_cat',
#  'feats_style_cat',
#  'feats_fits over eyeglasses_cat',
#  'feats_pocket_cat',
#  'feats_watch band material_cat',
#  'feats_labor_cat',
#  'feats_military style_cat',
#  'feats_gold karat_cat',
#  'feats_chain length_cat',
#  'feats_fit:_cat',
#  'feats_pronation_cat',
 'feats_case type_cat',
 'feats_cushioning_cat',
 'feats_is polarized_cat',
 'feats_band width_cat',
 'feats_model_cat',
 'feats_bed size_cat',
 'feats_shoe category_cat',
 'feats_wheeled_cat',
 'feats_polarized lenses_cat',
 'feats_front style_cat',
 'feats_fit_cat',
 'feats_casing_cat',
 'feats_rx-able_cat',
 'feats_combined shipping_cat',
 'feats_packageweight_cat',
 'feats_sports team_cat',
 'feats_primary color_cat',
 'feats_diamond color_cat',
 'feats_has adaptive lenses_cat',
 'feats_case tone_cat',
 'feats_safety features_cat',
 'feats_net weight_cat',
 'feats_power type_cat',
 'feats_sock size_cat',
 'feats_shoe height_cat',
 'feats_garment care_cat',
 'feats_package_cat',
 'feats_water resistant_cat',
 'feats_location - city/state_cat',
 'feats_материал_cat',
 'feats_terrain_cat',
 'feats_dimensions_cat',
 'feats_adjustable_cat',
 'feats_ul safety listing_cat',
 'feats_made in_cat',
 'feats_msrp_cat',
 'feats_number of pieces_cat',
 'feats_color/finish family_cat',
 'feats_number of pockets_cat',
 'feats_case material_cat',
 'feats_nickel free_cat',
 'feats_is portable_cat',
 'feats_adidas_cat',
 'feats_leather :_cat',
 'feats_video game platform_cat',
 'feats_has mercury_cat',
 'feats_removable liner_cat',
 'feats_thick, warm, cozy_cat',
 'feats_seller inventory number_cat',
 'feats_is wheeled_cat',
 'feats_fishing_cat',
 'feats_battery voltage_cat',
 'feats_disclosures_cat',
 'feats_series name_cat',
 'feats_variant group id_cat',
 'feats_inv_cat',
 'feats_fuel type_cat',
 'feats_item spec attr - style (ebay)_cat',
 'feats_stability_cat',
 'feats_removable hood_cat',
 'feats_item style_cat',
 'feats_band manufacturer_cat',
 'feats_bracelet style_cat',
 'feats_lens color_cat',
 'feats_sole_cat',
 'feats_boot height_cat',
 'feats_assembled in country of origin_cat',
 'feats_lens material_cat',
 'feats_recommended use_cat',
 'feats_country////region of manufacture_cat',
 'feats_country of origin assembly:_cat',
 'feats_diamond clarity_cat',
 'feats_recommended location_cat',
 'feats_necklace type_cat',
 'feats_size_cat',
 'feats_general warranty_cat',
 'feats_frame material_cat',
 'feats_to fit_cat',
 'feats_frame shape_cat',
 'feats_closure_cat',
 'feats_shipping weight_cat',
 'feats_cleaning, care & maintenance_cat',
 'feats_hat style_cat',
 'feats_has paper wood_cat',
 'feats_display technology_cat',
 'feats_free item_cat',
 'feats_heel height_cat',
 'feats_very popular bag now a days_cat',
 'feats_interior pockets_cat',
 'feats_chain included_cat',
 'feats_protection_cat',
 'feats_airport friendly_cat',
 'feats_leg_cat',
 'feats_contains batteries_cat',
 'feats_hands_cat',
 'feats_band length (in.)_cat',
 'feats_pant style_cat',
 'feats_main colour_cat',
 'feats_weather-resistant_cat',
 'feats_100% adidas authentic_cat',
 'feats_capacity_cat',
 'feats_hairstyle_cat',
 'feats_condition_cat',
 'feats_finish_cat',
 'feats_material composition_cat',
 'feats_number of batteries included_cat',
 'feats_material detail-1_cat',
 'feats_watch shape_cat',
 'feats_guaranteed authentic_cat',
 'feats_features_cat',
 'feats_movement_cat',
 'feats_item width_cat',
 'feats_autographed_cat',
 'feats_nike product #_cat',
 'feats_safety apparel product type_cat',
 'feats_designer_cat',
 'feats_target audience_cat',
 'feats_black_cat',
 'feats_ground_cat',
 'feats_construction_cat',
 'feats_sub style_cat',
 'feats_sports league_cat',
 'feats_width_cat',
 'feats_case type:_cat',
 'feats_lining material_cat',
 'feats_eyewear frame style_cat',
 'feats_expandable_cat',
 'feats_auto shut-off_cat',
 'feats_fabric content_cat',
 'feats_display_cat',
 'feats_leather grade_cat',
 'feats_waist size_cat',
 'feats_country of manufacture_cat',
 'feats_domestic shipping_cat',
 'feats_international shipping?_cat',
 'feats_subdial_cat',
 'feats_hardsided or softsided_cat',
 'feats_lens tint_cat',
 'feats_lens socket width_cat',
 'feats_sizebridge_cat',
 'feats_szie_cat',
 'feats_product name_cat',
 'feats_retail price_cat',
 'feats_number of items_cat',
 'feats_bridge size:_cat',
 'feats_smart watch_cat',
 'feats_country/ organization_cat',
 'feats_light weight_cat',
 'feats_case diameter_cat',
 'feats_item type_cat',
 'feats_height_cat',
 'feats_dial color_cat',
 'feats_sku#_cat',
 'feats_fine or fashion_cat',
 'feats_age gender group_cat',
 'feats_date first available_cat',
 'feats_chest size_cat',
 'feats_fabric material_cat',
 'feats_bridge_cat',
 'feats_quantity in set_cat',
 'feats_metal stamp_cat',
 'feats_productgroup_cat',
 'feats_kids backpacks_cat',
 'feats_parts_cat',
 'feats_digital camera_cat',
 'feats_color family_cat',
 'feats_motion control_cat',
 'feats_origin of components_cat',
 'feats_is recyclable_cat',
 'feats_resizable_cat',
 'feats_lining_cat',
 'feats_features & fastening_cat',
 'feats_insulation_cat',
 'feats_metal color_cat',
 'feats_colour code_cat',
 'feats_measurements:_cat',
 'feats_is lined_cat',
 'feats_shoe size_cat',
 'feats_location - country_cat',
 'feats_charger included_cat',
 'feats_product id_cat',
 'feats_style #_cat',
 'feats_lens type_cat',
 'feats_outer material_cat',
 'feats_label_cat',
 'feats_year_cat',
 'feats_assembled product weight_cat',
 'feats_product dimensions_cat',
 'feats_shoe width_cat',
 'feats_is orthopedic_cat',
 'feats_compass_cat',
 'feats_characteristics_cat',
 'feats_material type_cat',
 'feats_what includes_cat',
 'feats_hood_cat',
 'feats_conflict_cat',
 'feats_is waterproof_cat',
 'feats_includes_cat',
 'feats_spikes type_cat',
 'feats_season_cat',
 'feats_shoe closure_cat',
 'feats_clothing size type_cat',
 'feats_year made_cat',
 'feats_style number_cat',
 'feats_profession_cat',
 'feats_rise_cat',
 'feats_certifications and listings_cat',
 'feats_dial markers_cat',
 'feats_footwear type_cat',
 'feats_size/dimensions_cat',
 'feats_tactical_cat',
 'feats_authenticity_cat',
 'feats_length_cat',
 'feats_applicable_cat',
 'feats_shoe fastening type_cat',
 'feats_contained battery type_cat',
 'feats_inseam_cat',
 'feats_chronograph_cat',
 'feats_transactionid_cat',
 'feats_watch style_cat',
 'feats_shape_cat',
 'feats_fits model_cat',
 'feats_machine washable_cat',
 'feats_polarized?_cat',
 'feats_stock no._cat',
 'feats_jewelry setting_cat',
 'feats_manufacturer number_cat',
 'feats_work shoes_cat',
 'feats_looks best on these face shapes:_cat',
 'feats_style code_cat',
 'feats_water-resistant_cat',
 'feats_comfort technology_cat',
 'feats_batteries required_cat',
 'feats_number of heat settings_cat',
 'feats_isbn_cat',
 'feats_antiscratch lens coating_cat',
 'feats_style:_cat',
 'feats_country of origin - components_cat',
 'feats_type of sole_cat',
 'feats_era_cat',
 'feats_country of origin_cat',
 'feats_product #_cat',
 'feats_overall material_cat',
 'feats_pattern_cat',
 'feats_hammer loop_cat',
 'feats_item color_cat',
 'feats_boxed-product dimensions_cat',
 'feats_navy blue_cat',
 'feats_golf shoe type_cat',
 'feats_age end_cat',
 'feats_lens technology_cat',
 'feats_manufacturer_part_number_cat',
 'feats_item weight_cat',
 'feats_safety feature_cat',
 'feats_suitable for_cat',
 'feats_country of origin - assembly_cat',
 'feats_frame depth_cat',
 'feats_temple length_cat',
 'feats_frame style_cat',
 'feats_global composite sports type_cat',
 'feats_age_cat',
 'feats_purpose_cat',
 'feats_accessory type_cat',
 'feats_fit to_cat',
 'feats_impact resistant_cat',
 'feats_full product manual_cat',
 'feats_uv rating_cat',
 'feats_name_cat',
 'feats_elastic banded waist_cat',
 'feats_chest_cat',
 'feats_frame type_cat',
 'feats_lens width_cat',
 'feats_clothing type_cat',
 'feats_is water-resistant_cat',
 'feats_temple size:_cat',
 'feats_sku_cat',
 'feats_sleeve length_cat',
 'feats_handbag style_cat',
 'feats_heat zones_cat',
 'feats_eye size:_cat',
 'feats_auto_cat',
 'feats_fabric_cat',
 'feats_animal type_cat',
 'feats_band length_cat',
 'feats_colour_cat',
 'feats_gps_cat',
 'feats_charge time (minutes)_cat',
 'feats_assembled product dimensions (l x w x h)_cat',
 'feats_front_cat',
 'feats_product type_cat',
 'feats_color mapping_cat',
 'feats_clothing product type_cat',
 'feats_fabric type_cat',
 'feats_contains cubic zirconia_cat',
 'feats_multi pack indicator_cat',
 'feats_age range_cat',
 'feats_upper material_cat',
 'feats_use_cat',
 'feats_release_cat',
 'feats_vehicle type_cat',
 'feats_designed for_cat',
 'feats_shade_cat',
 'feats_clothing category_cat',
 'feats_stylenumber_cat',
 'feats_hunting_cat',
 'feats_pattern or design_cat',
 'feats_reinforced knee_cat',
 'feats_sleeve style_cat',
 'feats_band type_cat',
 'feats_manufacturer_cat',
 'feats_face shape_cat',
 'feats_cm_cat',
 'feats_count_cat',
 'feats_protects against_cat',
 'feats_ean_cat',
 'feats_country of origin components:_cat',
 'feats_nike id_cat',
 'feats_materials_cat',
 'feats_clasp type_cat',
 'feats_part number_cat',
 'feats_walmart no._cat',
 'feats_diameter_cat',
 'feats_weight_cat',
 'feats_power reserve_cat',
 'feats_leg opening:_cat',
 'feats_bridge width_cat',
 'feats_polarised_cat',
 'feats_hooded_cat',
 'feats_genre_cat',
 'feats_has expiration_cat',
 'feats_flame resistant_cat',
 'feats_case back_cat',
 'feats_place of origin_cat',
 'feats_toe type_cat',
 'feats_number of wheels_cat',
 'feats_wind resistant_cat',
 'feats_item package quantity_cat',
 'feats_picture_cat',
 'feats_heart rate monitor_cat',
 'feats_case finish_cat',
 'feats_box_cat',
 'feats_ring size_cat',
 'feats_neckline_cat',
 'feats_primary shelf id_cat',
 'feats_light transmission (vlt)_cat',
 'feats_theme_cat',
 'feats_type_cat',
 'feats_watch power source_cat',
 'feats_date_cat',
 'feats_audience_cat',
 'feats_character_cat',
 'feats_waist_cat',
 'feats_international shipping_cat',
 'feats_vendor description_cat',
 'feats_sizearm_cat',
 'feats_carats_cat',
 'feats_amazonbestsellersrank_cat',
 'feats_age group_cat',
 'feats_issued/ not-issued_cat',
 'feats_instrument_cat',
 'feats_item character_cat',
 'feats_age start_cat',
 'feats_type 2_cat',
 'feats_style name_cat',
 'feats_model no._cat',
 'feats_mirrored_cat',
 'feats_fits brands_cat',
 'feats_model number_cat',
 'feats_sock style_cat',
 'feats_dial material_cat',
 'feats_fastener_cat',
 'feats_is weather-resistant_cat',
 'feats_date first available at amazon.co.uk_cat',
 'feats_enhanced visibility (reflective)_cat',
 'feats_attachment_cat',
 'feats_battery type_cat',
 'feats_shirt size_cat',
 'feats_resolution_cat',
 'feats_part type_cat',
 'feats_resolution megapixels_cat',
 'feats_lined_cat',
 'feats_lens color family_cat',
 'feats_compatible devices_cat',
 'feats_number of compartments_cat',
 'feats_frame material:_cat',
 'feats_fabric care instructions_cat',
 'feats_fabrication_cat',
 'feats_sub type_cat',
 'feats_high visibility (ansi compliant)_cat',
 'feats_special features_cat',
 'feats_euro size_cat',
 'feats_eye protection type_cat',
 'feats_casual & dress shoe style_cat',
 'feats_case thickness_cat',
 'feats_shipping weight (in pounds)_cat',
 'feats_athlete_cat',
 'feats_jacket length_cat',
 'feats_closure style_cat',
 'feats_mpn#_cat',
 'feats_mpn_cat',
 'feats_weather resistant_cat',
 'feats_foot arch_cat',
 'feats_shipping to usa_cat',
 'feats_bezel_cat',
 'feats_atpv arc rating_cat',
 'feats_alarm_cat',
 'feats_date first available at amazon.ca_cat',
 'feats_manufacturer sku_cat',
 'feats_country of manufacturer_cat',
 'feats_feature_cat',
 'feats_metal type_cat',
 'feats_fastener type_cat',
 'feats_licensed_cat',
 'feats_wheel type_cat',
 'feats_item weight unit of measure_cat',
 'feats_arm_cat',
 'feats_chain/necklace length (in.)_cat',
 'feats_fits styles_cat',
 'feats_lens material:_cat',
 'feats_used with equipment type_cat',
 'feats_gemstone type_cat',
 'feats_polarized_cat',
 'feats_technician_cat',
 'feats_fits most screen size_cat',
 'feats_ring style_cat',
 'feats_batteries included?_cat',
 'feats_insulated_cat',
 'feats_uv protection_cat',
 'feats_watch case shape_cat',
 'feats_stone type_cat',
 'feats_batteries required?_cat',
 'feats_frame color_cat',
 'feats_age segment_cat',
 'feats_bridge/temple size:_cat',
 'feats_is dark sky-compliant_cat',
 'feats_sku number_cat',
 'feats_waterproof_cat',
 'feats_catalog_cat',
 'feats_bag size range_cat',
 'feats_mechanic_cat',
 'feats_main color_cat',
 'feats_fits models_cat',
 'feats_reinforced pockets_cat',
 'feats_is energy star-certified_cat',
 'feats_country/region of manufacture_cat',
 'feats_article_cat',
 'feats_fabric care_cat',
 'feats_clothing size_cat',
 'feats_wash_cat',
 'feats_best sellers rank_cat',
 'feats_crystal_cat',
 'feats_lens_cat',
 'feats_country//region of manufacture_cat',
 'feats_design_cat',
#  'feats_product in inches (l x w x h)_cat',
 'feats_occasion_cat',
 'feats_band color_cat',
 'feats_crown_cat',
 'feats_band material_cat']
# feats = set(feats)

model = RandomForestRegressor(max_depth=5, n_estimators=100)

result = run_model(feats, model)

(-57.668550384049524, 4.296491640539332)

In [59]:
X = df[feats].values
y = df[ 'prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2508 ± 0.0088,brand_cat
0.1095 ± 0.0080,feats_material_cat
0.0132 ± 0.0023,feats_brand_cat
0.0127 ± 0.0010,feats_weight_cat
0.0116 ± 0.0013,feats_fabric content_cat
0.0077 ± 0.0028,feats_adjustable_cat
0.0071 ± 0.0034,feats_resizable_cat
0.0056 ± 0.0006,feats_shoe category_cat
0.0045 ± 0.0003,feats_fabric material_cat
0.0035 ± 0.0003,feats_jacket length_cat


In [43]:
df [ df.brand == 'nike' ].features_parsed.sample(10).values

array([{'sport': 'soccer, unisex', 'main color': 'black orange white'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'manufacturer part number': '555447-570', 'gender': 'men', 'shoe category': 'athletic & sneakers', 'brand': 'nike', 'color': 'purple'},
       {'style': 'running, cross training', 'pattern': 'polka dot', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'brand': 'nike', 'fabric content': '100% fabric', 'color': 'wolf grey-white'},
       {'style': 'running, cross training', 'country/region of manufacture': 'vietnam', 'condition': 'new with box'},
       {'gender': 'men', 'color': 'black', 'condition': 'new with tags'},
       {'sport': 'baseball & softball', 'main color': 'black'},
       {'sport': 'basketball shoes', 'occasion': 'basketball shoes', 'material': 'mesh', 'gender': 'men', 'shoe size': '10 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.